### Fucking imports

In [ ]:
# System
import os, sys, io, glob, time

# Model
from models.uNet import *
from helpers.training_crossval import training
from helpers.cross_val import build_k_indices
from helpers.aerial_dataset import AerialDataset
from helpers.patched_aerial_dataset import PatchedAerialDataset
from helpers.image_helpers import imshow_tensor, imshow_tensor_gt, generate_predictions
from helpers.preprocessing.augmentation_config import ImageAugmentationConfig
from helpers.mask_to_submission import *
import random

# ML Library
import numpy as np
import torch, torchvision
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

# Image processing
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Reloading
%load_ext autoreload
%autoreload 2

### Fucking Constants

In [ ]:
TRAIN_IMAGE_DATA = './data/train/images/'
TRAIN_LABEL_DATA = './data/train/groundtruth/'
TEST_IMAGE_DATA = './data/test/images/'
TEST_LABEL_DATA = './data/train/predictions/'

SEED = 428796

BATCH_SIZE = 64
NUM_EPOCHS = 30
PATCH_SIZE = 80
TESTING_SIZE = 50

DATA_SIZE = 100
TRAINING_SIZE = 80

TEST_IMG_W, TEST_IMG_H, TEST_PATCH_SIZE_W, TEST_PATCH_SIZE_H = 608, 608, 16, 16
THRESHOLD = 0.25

CUDA = True
PERM = True

# np.random.seed(SEED)
# random.seed(SEED)
# if CUDA:torch.cuda.manual_seed_all(SEED)

dic = {"bsize" : BATCH_SIZE,
      "psize": PATCH_SIZE,
      "threshold" : THRESHOLD}
with open('./progress.txt', 'a') as file:
    file.writelines('\n' + "Running the code" + time.strftime("%d-%m-%Y-%h-%s") + '\n')
    file.writelines('Params : ' + str(dic))

### Fucking image augmentation

In [ ]:
aug_config = ImageAugmentationConfig()
aug_config.rotation(range(10, 180, 10))
#aug_config.edge()
#aug_config.blur()
#aug_config.contrast()

### Fucking Datasets

In [ ]:
K_FOLD = 1

In [ ]:
k_indices = build_k_indices(1, DATA_SIZE, K_FOLD)
datasets, dataloaders = [], []

for k in range(K_FOLD):
    print('Building fold {}'.format(k))
    # Get fold indices
    train_indices, val_indices = k_indices[k][1], k_indices[k][0]
    
    # Build datasets
    trainset = PatchedAerialDataset(TRAIN_IMAGE_DATA, TRAIN_LABEL_DATA, train_indices, PATCH_SIZE, aug_config)
    validationset = PatchedAerialDataset(TRAIN_IMAGE_DATA, TRAIN_LABEL_DATA, val_indices, PATCH_SIZE, aug_config)
    
    # Build data loaders
    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    validationloader = DataLoader(validationset, batch_size=1, shuffle=False)
    
    datasets.append({'train':trainset, 'val':validationset})
    dataloaders.append({'train':trainloader, 'val':validationloader})

### Fucking uNet

In [ ]:
models = []
for k in range(K_FOLD):
    model = UNet().cuda(1)
    dic = {
        'model': model,
        'criterion': torch.nn.BCEWithLogitsLoss().cuda(1),
        'optimizer': optim.Adam(model.parameters(), lr=0.001)
    }
    
    models.append(dic)

### Fucking Training

In [ ]:
acc, best_model = training(NUM_EPOCHS, models, datasets, dataloaders, PATCH_SIZE, K_FOLD, cuda=CUDA)

### Fucking plotting the fucking accuracy vs epoch fucking plot

In [ ]:
# Print accuracy curve
fig = plt.figure()
x = np.arange(0, NUM_EPOCHS)
plt.plot(x, acc, '--')
fig.suptitle(time.strftime("%d-%m-%Y" ))

plt.xlabel('epoch')
plt.ylabel('accuracy (validation data)')
plt.savefig('./plots/epoch_accuracy' + time.strftime("%d-%m-%Y-%h-%s") + '.png', dpi=fig.dpi)
plt.show()

### Time to get the FUCKING testing data

In [ ]:
test_indices = np.arange(1, TESTING_SIZE+ 1,1)
testset = AerialDataset(TEST_IMAGE_DATA, test_indices, aug_config)
testloader = DataLoader(testset, batch_size=1, shuffle=False)

### Feed the motherfucking NN testing data

In [ ]:
truth = []
predicted_labels = []
model.eval()
for data in testloader:
    patches = data
    
    if CUDA:
        patches = patches.cuda(1)
        
    patches = Variable(patches)

    # Feed the neural net with the patches
    outputs = model(patches)
    predicted = np.rint(outputs.squeeze().data.cpu().numpy())
    predicted_labels.append(predicted)
    truth.append(patches)

### How about looking at one output ?

In [ ]:
image_id = 40

imshow_tensor(torchvision.utils.make_grid(truth[image_id].cpu().data, normalize=True))
plt.imshow(predicted_labels[image_id])

In [ ]:
def labels_to_patches(image, imgw, imgh, pw, ph, threshold):
    array_labels = np.zeros([imgh, imgh])
    for i in range(0,imgw,pw):
        for j in range(0,imgw,ph):
            mean = np.mean(image[i:i+pw, j:j+ph])
            if mean > threshold: array_labels[i:i+pw, j:j+ph] = 1
            else: array_labels[i:i+pw, j:j+ph] = 0
    return array_labels

In [ ]:
img_patches = []
for labels in predicted_labels:
    img_patches.append(labels_to_patches(labels, TEST_IMG_W, TEST_IMG_H, TEST_PATCH_SIZE_W, TEST_PATCH_SIZE_H, THRESHOLD))

In [ ]:
plt.imshow(np.concatenate([predicted_labels[image_id],img_patches[image_id]], 1))

In [ ]:
generate_predictions(TESTING_SIZE, img_patches, concatenate = False)

In [ ]:
generate_submission_csv()